In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
dataset = pd.read_csv('Full.csv')

features = ['Soil Type', 'Water Availability', 'Area Size', 'Season', 'Temperature', 'Rainfall', 'Crop Cycle']
target_crop = 'Crop Recommendation'
target_yield = 'Yield Prediction'
target_fertilizer = 'Fertilizer Recommendation'
target_pesticide = 'Pesticide Recommendation'

In [3]:
X = dataset[features]
y_crop = dataset[target_crop]
y_yield = dataset[target_yield]
y_fertilizer = dataset[target_fertilizer]
y_pesticide = dataset[target_pesticide]

In [4]:
X_train, X_test, y_crop_train, y_crop_test, y_yield_train, y_yield_test, y_fertilizer_train, y_fertilizer_test, y_pesticide_train, y_pesticide_test = train_test_split(
    X, y_crop, y_yield, y_fertilizer, y_pesticide, test_size=0.2, random_state=42)

In [5]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Area Size', 'Temperature']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Soil Type', 'Water Availability', 'Season', 'Rainfall', 'Crop Cycle'])
    ])


In [6]:
pipeline_crop = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Yield Prediction model
pipeline_yield = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

# Fertilizer Recommendation model
pipeline_fertilizer = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Pesticide Recommendation model
pipeline_pesticide = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

In [7]:
pipeline_crop.fit(X_train, y_crop_train)
pipeline_yield.fit(X_train, y_yield_train)
pipeline_fertilizer.fit(X_train, y_fertilizer_train)
pipeline_pesticide.fit(X_train, y_pesticide_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Area Size', 'Temperature']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Soil Type',
                                                   'Water Availability',
                                                   'Season', 'Rainfall',
                                                   'Crop Cycle'])])),
                ('classifier', RandomForestClassifier())])

In [8]:
crop_predictions = pipeline_crop.predict(X_test)
yield_predictions = pipeline_yield.predict(X_test)
fertilizer_predictions = pipeline_fertilizer.predict(X_test)
pesticide_predictions = pipeline_pesticide.predict(X_test)

In [9]:
print("Crop Recommendation Accuracy:", accuracy_score(y_crop_test, crop_predictions))
print("Yield Prediction MSE:", mean_squared_error(y_yield_test, yield_predictions))
print("Fertilizer Recommendation Accuracy:", accuracy_score(y_fertilizer_test, fertilizer_predictions))
print("Pesticide Recommendation Accuracy:", accuracy_score(y_pesticide_test, pesticide_predictions))


Crop Recommendation Accuracy: 0.635
Yield Prediction MSE: 0.02314796000000024
Fertilizer Recommendation Accuracy: 0.89
Pesticide Recommendation Accuracy: 0.85


In [10]:
def make_predictions(new_data):
    crop_pred = pipeline_crop.predict(new_data)
    yield_pred = pipeline_yield.predict(new_data)
    fertilizer_pred = pipeline_fertilizer.predict(new_data)
    pesticide_pred = pipeline_pesticide.predict(new_data)
    return crop_pred, yield_pred, fertilizer_pred, pesticide_pred

In [11]:
new_example = pd.DataFrame({
    'Soil Type': ['Sandy'],
    'Water Availability': ['low'],
    'Area Size': [420],
    'Season': ['summer'],
    'Temperature': [30],
    'Rainfall': ['high'],
    'Crop Cycle': ['short term']
})
predictions = make_predictions(new_example)
print("Predictions for new example:", predictions)

Predictions for new example: (array(['Potato'], dtype=object), array([5.617]), array(['Nitrogen-rich fertilizer'], dtype=object), array(['Organic pesticide'], dtype=object))


In [12]:
import joblib

# Save the models
joblib.dump(pipeline_crop, 'pipeline_crop.pkl')
joblib.dump(pipeline_yield, 'pipeline_yield.pkl')
joblib.dump(pipeline_fertilizer, 'pipeline_fertilizer.pkl')
joblib.dump(pipeline_pesticide, 'pipeline_pesticide.pkl')


['pipeline_pesticide.pkl']